# Quantization of models

<a target="_blank" href="https://colab.research.google.com/github/toelt-llc/HSLU-WSCS_2025/blob/master/06%20-%20Quantization_of_models_Complete_examples.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

(C) Umberto Michelucci

umberto.michelucci@toelt.ai

www.toelt.ai


In [1]:
# !pip3 uninstall tensorflow
# !pip3 install tf-nightly

In [2]:
#
# DON'T USE THE FOLLOWING MAGIC COMMAND IN CASE YOU ARE INSTALLING TF-NIGHTLY.
# IT WILL USE THE WRONG VERSION OF TENSORFLOW.
#
#try:
  # %tensorflow_version only exists in Colab.
  #%tensorflow_version 2.x
#except Exception:
#  pass


import tensorflow as tf

2025-03-27 08:06:28.577250: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print(tf.__version__)

2.16.2


In [4]:
import sys
import os

if sys.version_info.major >= 3:
    import pathlib
else:
    import pathlib2 as pathlib

# Add `models` to the python path.
models_path = os.path.join(os.getcwd(), "models")
sys.path.append(models_path)

In [5]:
saved_models_root = "/tmp/mnist_saved_model"

#  Simple MNIST Model

## Fitting and saving the model

In [6]:
import tensorflow.keras as keras
import numpy as np

import matplotlib.pyplot as plt

We first import the `MNIST` dataset with the `tf.keras.datasets`. Note that we reshape the input into a `(60000, 784)` array.

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.shape)

x_train = x_train.reshape((60000,784))

(60000, 28, 28)


We build a very dumb network with just one hidden layer with 128 neurons with `relu` activation functions. The output layer has 10 neurons (since we want to classify 10 classes) and then a `softmax` activation function.

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
         optimizer=keras.optimizers.Adam(),
         metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8741 - loss: 0.4365
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9666 - loss: 0.1174
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9763 - loss: 0.0784
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9824 - loss: 0.0605
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9862 - loss: 0.0451


The following cell will save the model. We will use this saved version in the next sections.

In [10]:
export_dir = "/tmp/mnist"
tf.saved_model.save(model, export_dir)

TypeError: this __dict__ descriptor does not support '_DictWrapper' objects

# Plain Conversion (without quantization)

As a first step, we can simply convert the model without any quantization. Basically we simply change its format to `TFLite` but we will not change any weight.

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.experimental_new_converter = True # otherwise you get a warning
tflite_model = converter.convert()

In [ ]:
tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
tflite_model_file = tflite_models_dir/"mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model),

(408768,)

In [ ]:
from google.colab import files
files.download(tflite_models_dir/"mnist_model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Convert with weight quantization to 8-bits

The following `optimizations` will quantize the weights to 8-bits. So the converted model will have a reduced size as you can see below in the last cell of this section. Note that going from 32-bit to 8-bit will reduce the model size of 1/4.

In [ ]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

In [ ]:
tflite_model_quant = converter.convert()
tflite_model_quant_file = tflite_models_dir/"mnist_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

104024

In [ ]:
from google.colab import files
files.download(tflite_models_dir/"mnist_model_quant.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls -lh {tflite_models_dir}

total 504K
-rw-r--r-- 1 root root 102K Sep  4 14:12 mnist_model_quant.tflite
-rw-r--r-- 1 root root 400K Sep  4 14:11 mnist_model.tflite


# Evaluation of accuracy of the models

In [ ]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [ ]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [ ]:
image_1 = x_train[0].astype(np.float32)
image_1 = image_1.reshape(1,784)

interpreter.set_tensor(interpreter.get_input_details()[0]["index"], image_1)
interpreter.invoke()
predictions = interpreter.get_tensor(
    interpreter.get_output_details()[0]["index"])

In [ ]:
print(np.argmax(predictions))
print(y_train[0])

5
5


Now test the quantized model (using the uint8 data):

In [ ]:
image_1 = x_train[0].astype(np.float32)
image_1 = image_1.reshape(1,784)

interpreter_quant.set_tensor(
    interpreter_quant.get_input_details()[0]["index"], image_1)
interpreter_quant.invoke()
predictions = interpreter_quant.get_tensor(
    interpreter_quant.get_output_details()[0]["index"])

In [ ]:
print(np.argmax(predictions))
print(y_train[0])

5
5
